In [1]:
import os
os.chdir("../")
os.getcwd()

'/home/izam/coding/hackathon'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PromptingConfig:
    root_dir: Path
    prompt: list
    model_name: str
    response_file_name: str
    candidates_file_name: str
    credentials: Path
    generation_config: dict
    project_name: str
    project_location: str

In [3]:
from gemInsights.constants import *
from gemInsights.utils.common import read_yaml, create_directories, load_json

In [4]:
#udpate configuration manager
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        credentials_file_path = CREDENTIALS_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.credentials = credentials_file_path
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])


    def get_promting_config(self, prompt) -> PromptingConfig:
        config = self.config.prompting
        
        create_directories([config.root_dir])

        prompting_config = PromptingConfig(
            root_dir=config.root_dir,
            prompt=prompt,
            model_name=config.model_name, 
            response_file_name=config.response_file_name,
            candidates_file_name=config.candidates_file_name,
            credentials=self.credentials,
            generation_config=dict(self.params.generation_config),
            project_name=config.project_name,
            project_location=config.project_location            
        )

        return prompting_config

In [5]:
from google.cloud import aiplatform
from gemInsights import logger
import os
import base64
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
from gemInsights.utils.common import load_json, save_json

In [15]:
class Prompting:
    def __init__(self, config: PromptingConfig):
        self.config = config

    def get_response(self):
        aiplatform.init(
            project = self.config.project_name,
            location= self.config.project_location,
        )
        logger.info(f"Google cloud project name - {self.config.project_name}")
        
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = self.config.credentials
        logger.info("loaded the google cloud credentials")

        model = GenerativeModel(self.config.model_name)
        logger.info(f"using the model - {self.config.model_name}")

        logger.info(f"generating response with config - {self.config.generation_config}")
        responses = model.generate_content(
            self.config.prompt,
            generation_config=self.config.generation_config,
            )
        
        save_json(path=os.path.join(self.config.root_dir, self.config.response_file_name), data={"response": responses.text})
        logger.info(responses.text)

In [16]:
import base64
img = open("./research/Rating/Dist_Plots_Cats.jpg", "rb").read()
img_bytes = Part.from_data(base64.b64decode(base64.encodebytes(img)), mime_type="image/jpeg")

In [17]:
try:
    config = ConfigurationManager()
    prompting_config = config.get_promting_config(["explain this image", img_bytes])
    prompting_config = Prompting(config=prompting_config)
    prompting_config.get_response()
except Exception as e:
    raise e

[2023-12-21 11:41:16,654: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-21 11:41:16,656: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-21 11:41:16,657: INFO: common: created directory at: artifacts]
[2023-12-21 11:41:16,657: INFO: common: created directory at: artifacts/prompting]
[2023-12-21 11:41:16,658: INFO: 2160553634: Google cloud project name - ultra-heading-407815]
[2023-12-21 11:41:16,658: INFO: 2160553634: loaded the google cloud credentials]
[2023-12-21 11:41:16,658: INFO: 2160553634: using the model - gemini-pro-vision]
[2023-12-21 11:41:16,659: INFO: 2160553634: generating response with config - {'max_output_tokens': 2048, 'temperature': 0.4, 'top_p': 1, 'top_k': 32}]
[2023-12-21 11:41:27,856: INFO: common: json file saved at: artifacts/prompting/response.json]
[2023-12-21 11:41:27,856: INFO: 2160553634:  The image shows the distribution of customers, gender, branch, city, product line, payment, and date.

The distribution o